# 3 - Частеричная разметка как классификация токенов

Основано на [туториале от bentrevett](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1_bilstm.ipynb).
Русскоязычные данные взяты из [корпуса "Тайга"](https://tatianashavrina.github.io/taiga_site).

## Введение

Все задачи обработки текстов (и других последовательностей) можно свести к пяти группам в зависимости от того, что подается на вход и что ожидается на выходе.

![](assets/sequence_tasks.jpeg?raw=1)
*Источник: [Andrej Karpathy blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*

До этого мы с вами сталкивались только с задачей классификации всей последовательности.

В данном туториале мы посмотрим на то, как построить модель с использованием рекуррентных нейронных сетей (а именно двунаправленной LSTM), которая классифицирует каждый элемент входной последовательности. Как пример подобной задачи мы рассмотрим частеричную разметку (part-of-speech (POS) tagging). То же самое может быть применено к задаче выделения именованных сущностей, где для каждого слова генерируется тип сущности (если оно ею является).

![](assets/pos_tagging_example.png?raw=1)
*Источник: [курс Павла Браславского](https://stepik.org/course/1233)*

## Подготовка данных

Сначала импортируем необходимые модули.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from datasets import Dataset, DatasetDict

import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import sys
import time
import random

Установим единый сид, чтобы результаты были воспроизводимыми.

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Сначала прочитаем наши данные.
Это предложения из статей научно-популярного интернет-издания N+1, которые были размечены тегами UDPOS.
Данные находятся в директории `data/pos_nplus1`.
Каждое отдельное слово (токен) и соответствующий тег разделенные символом табуляции находятся на отдельной строке.

Заведем отдельные списки для токенов и тегов, а затем обернем их в объект класса `Dataset`.
Также предусмотрим возможность привести все токены к нижнему регистру.
Это может сказаться на качестве модели, но уменьшит её размер.

In [ ]:
def read_dataset(file_path, lower=True):
    tokens, pos_tags = [], []

    with open(file_path) as f:
        cur_tokens, cur_tags = [], []
        for line in f:
            line = line.strip()
            if not line:
                if cur_tokens:
                    tokens.append(cur_tokens)
                    pos_tags.append(cur_tags)
                    cur_tokens, cur_tags = [], []

                continue

            token, tag = line.split('\t')
            if lower:
                token = token.lower()
            cur_tokens.append(token)
            cur_tags.append(tag)

        if cur_tokens:
            tokens.append(cur_tokens)
            pos_tags.append(cur_tags)

    return Dataset.from_dict({'tokens': tokens, 'pos_tags': pos_tags})

In [ ]:
TEXT_LOWER = True

Посмотрим на пример объекта класса `Dataset`.

In [ ]:
d = read_dataset('data/pos_nplus1/train.txt', lower=TEXT_LOWER)

Объект содержит два поля `tokens` и `pos_tags`. В каждом поле прочитанные нами списки токенов и тегов.

In [ ]:
d

In [ ]:
d['tokens'][0]

In [ ]:
d['pos_tags'][0]

In [ ]:
for token, tag in zip(d['tokens'][0], d['pos_tags'][0]):
    print(f'{token}\t\t{tag}')

Посмотрим на распределение тегов в нашей тренировочной выборке.

In [ ]:
tags_counter = Counter()

for tags in d['pos_tags']:
    tags_counter.update(tags)

In [ ]:
n_train_tags = sum(tags_counter.values())

for tag, tag_count in tags_counter.most_common():
    print(f'{tag}\t{tag_count / n_train_tags * 100: .2f}%')

Прочитаем все выборки нашего датасета и обернем их в объект класса `DatasetDict`. По сути это словарь для датасетов, позволяющий сразу применять действия к нескольким объектам `Dataset`.

In [ ]:
data = DatasetDict()

for split_name in ['train', 'validation', 'test']:
    data[split_name] = read_dataset(f'data/pos_nplus1/{split_name}.txt', lower=TEXT_LOWER)

In [ ]:
data

Теперь построим словари (vocabulary) для наших полей. Словарь в данном случае -- это сопоставление каждому токену (тегу, лейблу) уникального индекса.

Словари строятся на основе тренировочной выборки, из-за чего на этапе валидации/эксплуатации могут встречаться незнакомые токены. Такие токены будут заменены на специальный токен `<unk>`. Чтобы снизить количество параметров и подготовить модель к встрече с незнакомыми словами мы установим `min_freq = 2`, из-за чего в словарь модели будут добавляться токены, которые встречаются минимум 2 раза. Также в наши словари будет добавлен специальный токен `<pad>`, который пригодится нам позднее.

In [ ]:
MIN_FREQ = 2

tokens_vocab = torchtext.vocab.build_vocab_from_iterator(data['train']['tokens'], min_freq=MIN_FREQ,
                                                         specials=['<unk>', '<pad>'])

tags_vocab = torchtext.vocab.build_vocab_from_iterator(data['train']['pos_tags'], specials=['<pad>'])

Посмотрим на размеры словарей и примеры.

In [ ]:
len(tokens_vocab), len(tags_vocab)

In [ ]:
tokens_vocab['ученые']

In [ ]:
tags_vocab['VERB']

Укажем для словаря токенов, чтобы при появлении незнакомого слова возвращался индекс токена `<unk>`.

In [ ]:
# tokens_vocab['blablabla']

In [ ]:
tokens_vocab.set_default_index(tokens_vocab['<unk>'])

In [ ]:
tokens_vocab['blablabla'], tokens_vocab['<unk>']

Мы можем получить полное сопостовление между токенами (тегами) и их индексами в словаре.

In [ ]:
idx_to_tag = tags_vocab.vocab.get_itos()
idx_to_tag

In [ ]:
tag_to_idx = tags_vocab.vocab.get_stoi()
tag_to_idx

Но удобнее пользоваться готовыми функциями, для преобразования нескольких токенов (тегов) в индексы и обратно.

In [ ]:
token_idxs_example = tokens_vocab.forward(data['train']['tokens'][42])
tag_idxs_example = tags_vocab.forward(data['train']['pos_tags'][42])

In [ ]:
print(token_idxs_example)

In [ ]:
print(tag_idxs_example)

In [ ]:
print(tokens_vocab.lookup_tokens(token_idxs_example))
print(data['train']['tokens'][42])

In [ ]:
print(tags_vocab.lookup_tokens(tag_idxs_example))
print(data['train']['pos_tags'][42])

Теперь, наконец, переведем все наши данные в числовой формат.

In [ ]:
def numericalize_data(example):
    token_idxs = tokens_vocab.forward(example['tokens'])
    tag_idxs = tags_vocab.forward(example['pos_tags'])
    return {'token_idxs': token_idxs, 'tag_idxs': tag_idxs}

Применяем функцию `numericalize_data` ко всем данным, удаляем ненужные колонки и переводим данные в тип `torch.Tensor`.

In [ ]:
transformed_data = data.map(numericalize_data, remove_columns=['tokens', 'pos_tags']).with_format(type='torch')

In [ ]:
transformed_data

In [ ]:
transformed_data['train']['token_idxs'][0]

In [ ]:
transformed_data['train']['tag_idxs'][0]

Последний этап предобработки данных -- это создание итераторов, которые будут перемешивать данные и делить их на батчи. За это отвечают объекты класса `DataLoader`. При создании итераторов будем передавать функцию `collate_batch`, которая принимает на вход фрагмент датасета и строит два списка: список токенов (точнее их индексов) и список тегов.

В то время как предложения обычно имеют разную длину, входные и выходные последовательности  представлены в виде трехмерного тензора. Поэтому все короткие предложения в батче приводятся к длине самого длинного предложения путем добавления токена `<pad>` (точнее его индекса).

In [ ]:
def collate_batch(batch):
    batch_tokens = [example['token_idxs'] for example in batch]
    batch_tags = [example['tag_idxs'] for example in batch]
    batch_tokens = nn.utils.rnn.pad_sequence(batch_tokens, padding_value=tokens_vocab['<pad>'], batch_first=True)
    batch_tags = nn.utils.rnn.pad_sequence(batch_tags, padding_value=tags_vocab['<pad>'], batch_first=True)
    batch = {'token_idxs': batch_tokens,
             'tag_idxs': batch_tags}
    return batch

In [ ]:
BATCH_SIZE = 32

train_dataloader = torch.utils.data.DataLoader(transformed_data['train'],
                                               batch_size=BATCH_SIZE,
                                               collate_fn=collate_batch,
                                               shuffle=True)

validation_dataloader = torch.utils.data.DataLoader(transformed_data['validation'],
                                                    batch_size=BATCH_SIZE,
                                                    collate_fn=collate_batch)

test_dataloader = torch.utils.data.DataLoader(transformed_data['test'],
                                              batch_size=BATCH_SIZE,
                                              collate_fn=collate_batch)

Посмотрим на пример батча:

In [ ]:
for batch in train_dataloader:
    break

In [ ]:
batch

Или так:

In [ ]:
next(iter(train_dataloader))

## Строим модель

Следующий шаг -- построить нашу модель. Модель будет включать два слоя двунаправленной (значит обрабатывать текст справа налево и слева направо) рекуррентной нейронной сети.

При этом в качестве рекуррентного слоя будет выступать не простая RNN, как мы реализовывали в прошлом ноутбуке, а LSTM. Основным недостатком классической RNN является то, что она обладает высокой забывчивостью. Для решения этой проблемы в LSTM был реализован дополнительный вектор состояния ячейки, который сеть на каждом шаге обновляет.

![](assets/forgetting.jpg)
*Источник: [LSTM – сети долгой краткосрочной памяти](https://habr.com/ru/companies/wunderfund/articles/331310/)*


![](assets/lstm.png)
*Источник: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)*

![](assets/pos-bidirectional-lstm.png?raw=1)

Модель получает на вход последовательность индексов токенов $X = \{x_1, x_2,...,x_T\}$, и проводит их через слой эмбеддингов $e$. Таким обазом получаем набор эмбеддингов для каждого слова: $e(X) = \{e(x_1), e(x_2), ..., e(x_T)\}$.

Эти эмбеддинги обрабатываются по одному в один момент времени: по прямой и обратной LSTM. Прямая LSTM обрабатывает последовательность слева направо, а обратная -- справо налево. То есть первый токен прямой LSTM -- $x_1$, первый токен обратной -- $x_T$.

Также LSTM получает а вход скрытое состояние $h$ и состояние ячейки $c$, полученные после обработки предыдущего слова.

$$h^{\rightarrow}_t = \text{LSTM}^{\rightarrow}(e(x^{\rightarrow}_t), h^{\rightarrow}_{t-1}, c^{\rightarrow}_{t-1})$$
$$h^{\leftarrow}_t=\text{LSTM}^{\leftarrow}(e(x^{\leftarrow}_t), h^{\leftarrow}_{t-1}, c^{\leftarrow}_{t-1})$$

После того, как последовательность была полностью обработана, скрытое состояние и состояние ячейчки передаются в следующий слой LSTM.

Начальные значения $h_0$ и $c_0$ инициализируются тензором из нулей.


Все скрытыте состояния после прямого и обратного прохода на последнем слое LSTM конкатенируются: $H = \{h_1, h_2, ... h_T\}$, где $h_1 = [h^{\rightarrow}_1;h^{\leftarrow}_T]$, $h_2 = [h^{\rightarrow}_2;h^{\leftarrow}_{T-1}]$ и т.д. и подаются на вход линейному слою $f$, который используется для предсказания того, какой тег соответствует токену: $\hat{y}_t = f(h_t)$.

В процессе тренировки модели мы будем сравнивать наши предсказанные теги $\hat{Y}$ с тегами из датасета $Y$, вычислять значение функции ошибки, вычислять градиенты и обновлять наши параметры.

`nn.Embedding` слой эмбеддингов, input_dim которого должен быть равен размеру словаря, а embedding_dim является подбираемым гиперпараметром. Также мы сообщаем слою индекс <pad> токена, чтобы не обновлять его эмбеддинг.

`nn.LSTM` слой LSTM. Если у нас несколько слоев LSTM, то между ними применяется дропаут.

`nn.Linear` определяет линейный слой, который делает предсказания с использованием выходов из LSTM. Если из LSTM мы ожидаем скрытые состояния после прямого и обратного прохода, то вход линейного слоя удваивается. Выход из линейного слоя должен быть равен размеру словаря тегов.

Также в качестве регуляризации в процессе обучения мы применяем слой `nn.Dropout` к эмбеддингам и выходам из LSTM.

In [ ]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers = n_layers,
                            bidirectional = bidirectional,
                            dropout = dropout if n_layers > 1 else 0)

        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        #pass text through embedding layer
        embedded = self.dropout(self.embedding(text))

        #embedded = [sent len, batch size, emb dim]

        #pass embeddings into LSTM
        outputs, (hidden, cell) = self.lstm(embedded)

        #outputs holds the backward and forward hidden states in the final layer
        #hidden and cell are the backward and forward hidden and cell states at the final time-step

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        #we use our outputs to make a prediction of what the tag should be
        predictions = self.fc(self.dropout(outputs))

        #predictions = [sent len, batch size, output dim]

        return predictions

## Обучаем модель

In [ ]:
INPUT_DIM = len(tokens_vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(tags_vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
TOKEN_PAD_IDX = tokens_vocab['<pad>']

model = BiLSTMPOSTagger(INPUT_DIM,
                        EMBEDDING_DIM,
                        HIDDEN_DIM,
                        OUTPUT_DIM,
                        N_LAYERS,
                        BIDIRECTIONAL,
                        DROPOUT,
                        TOKEN_PAD_IDX)

In [ ]:
model

In [ ]:
for p in model.parameters():
    print(p)
    break

Инициализируем веса из нормального распределения.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

model.apply(init_weights)

Считаем, сколько параметров в нашей модели:

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'В модели {count_parameters(model):,} обучаемых параметров')

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TAG_PAD_IDX = tags_vocab['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

In [ ]:
def train(model, dataloader, optimizer, criterion, device, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in tqdm(dataloader, desc='training...', file=sys.stdout):
        text = batch['token_idxs'].to(device)
        tags = batch['tag_idxs'].to(device)

        optimizer.zero_grad()

        #text = [sent len, batch size]

        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]

        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return (epoch_loss / len(dataloader), epoch_acc / len(dataloader))

In [ ]:
def evaluate(model, dataloader, criterion, device, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in tqdm(dataloader, desc='evaluating...', file=sys.stdout):

            text = batch['token_idxs'].to(device)
            tags = batch['tag_idxs'].to(device)

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return (epoch_loss / len(dataloader), epoch_acc / len(dataloader))

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    _, _ = train(model, train_dataloader, optimizer, criterion, device, TAG_PAD_IDX)
    epoch_train_loss, epoch_train_acc = evaluate(model, train_dataloader, criterion, device, TAG_PAD_IDX)
    epoch_valid_loss, epoch_valid_acc = evaluate(model, validation_dataloader, criterion, device, TAG_PAD_IDX)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if epoch_valid_loss < best_valid_loss:
        best_valid_loss = epoch_valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {epoch_train_loss:.3f} | Train Acc: {epoch_train_acc*100:.2f}%')
    print(f'\t Val. Loss: {epoch_valid_loss:.3f} |  Val. Acc: {epoch_valid_acc*100:.2f}%')

## Оцениваем модель на различные данных

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_dataloader, criterion, device, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

In [ ]:
def tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, lower=True):

    model.eval()

    if isinstance(sentence, str):
        tokens = word_tokenize(sentence, language='russian')
    else:
        tokens = sentence

    if lower:
        tokens = [token.lower() for token in tokens]

    numericalized_tokens = tokens_vocab.forward(tokens)

    unk_idx = tokens_vocab['<unk>']

    unks = [token for token, token_idx in zip(tokens, numericalized_tokens) if token_idx == unk_idx]

    token_tensor = torch.LongTensor(numericalized_tokens)

    token_tensor = token_tensor.unsqueeze(-1).to(device)

    predictions = model(token_tensor)

    top_predictions = predictions.argmax(-1)

    predicted_tags = tags_vocab.lookup_tokens(top_predictions.cpu().numpy())

    return tokens, predicted_tags, unks

In [ ]:
example_index = 1

sentence = data['test']['tokens'][example_index]
actual_tags = data['test']['pos_tags'][example_index]

print(sentence)

In [ ]:
tokens, pred_tags, unks = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)

print(unks)

In [ ]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

In [ ]:
all_actual_tags = np.hstack(data['test']['pos_tags'])
all_predicted_tags = np.array([])

for sentence in tqdm(data['test']['tokens']):
    _, pred_tags, _ = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)
    all_predicted_tags = np.hstack((all_predicted_tags, pred_tags))

In [ ]:
len(all_actual_tags), len(all_predicted_tags)

In [ ]:
labels = tags_vocab.get_itos()
labels.remove('<pad>')

In [ ]:
cm = confusion_matrix(all_actual_tags, all_predicted_tags, labels=labels, normalize='true') # , normalize='pred'

In [ ]:
cm

In [ ]:
cmd_obj = ConfusionMatrixDisplay(cm, display_labels=labels)
cmd_obj.plot(include_values=False, xticks_rotation='vertical')

In [ ]:
sentence = 'Сегодня анализ текста применяется во многих областях нашей жизни — от абсолютно повседневных вещей \
            вроде поисковых систем и Гугл-переводчика до нейросетей.'

In [ ]:
tokens, pred_tags, unks = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)

print(unks)

In [ ]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, pred_tags):
    print(f"{tag}\t\t{token}")

In [ ]:
sentence = 'Мистер Шерлок Холмс, имевший обыкновение вставать очень поздно, \
            за исключением тех нередких случаев, когда вовсе не ложился спать, сидел за завтраком.'

In [ ]:
tokens, pred_tags, unks = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)

print(unks)

In [ ]:
print("Pred. Tag\tToken\n")

for token, tag in zip(tokens, pred_tags):
    print(f"{tag}\t\t{token}")

In [ ]:
other_domain = read_dataset('data/pos_stihiru/test.txt', TEXT_LOWER)
other_domain

In [ ]:
transformed_other = other_domain.map(numericalize_data, remove_columns=['tokens', 'pos_tags']).with_format(type='torch')
transformed_other

In [ ]:
other_dataloader = torch.utils.data.DataLoader(transformed_other,
                                              batch_size=BATCH_SIZE,
                                              collate_fn=collate_batch)

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

other_loss, other_acc = evaluate(model, other_dataloader, criterion, device, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

In [ ]:
example_index = 1

sentence = other_domain['tokens'][example_index]
actual_tags = other_domain['pos_tags'][example_index]

print(sentence)

In [ ]:
tokens, pred_tags, unks = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)

print(unks)

In [ ]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

In [ ]:
all_actual_tags = np.hstack(other_domain['pos_tags'])
all_predicted_tags = np.array([])

for sentence in tqdm(other_domain['tokens']):
    _, pred_tags, _ = tag_sentence(model, device, sentence, tokens_vocab, tags_vocab, TEXT_LOWER)
    all_predicted_tags = np.hstack((all_predicted_tags, pred_tags))

In [ ]:
len(all_actual_tags), len(all_predicted_tags)

In [ ]:
labels = tags_vocab.get_itos()
labels.remove('<pad>')

In [ ]:
cm = confusion_matrix(all_actual_tags, all_predicted_tags, labels=labels, normalize='true')

In [ ]:
cmd_obj = ConfusionMatrixDisplay(cm, display_labels=labels)
cmd_obj.plot(include_values=False, xticks_rotation='vertical')